<h2>Desafio 4 - Trabajo Final</h2>

<h3>Grupo 7</h3>
<ul>
    <li>Ignacio Mendieta</li>
    <li>Laura Jazmín Chao</li>
    <li>Juan Nicolás Capistrano</li>
    <li>Betiana Srur</li>
    <li>Marcelo Carrizo</li>
    
</ul>
<h3>Ejemplo de cliente para predicción de consumo de CO2

<a id="section_toc"></a> 
<h2> Tabla de Contenidos </h2>

[Librerías](#section_import)

[Entrenamiento del model](#section_train)

[Visualización de coeficientes](#section_coef)

[Solicitud de predicción](#section_predict)

[Disponibilización del servidor](#section_server)

<a id="section_import"></a> 
<h3>Librerías</h3>

[volver a TOC](#section_toc)

In [23]:
# Algunas de las de siempre
import numpy as np
import pandas as pd

# Las que vamos a necesitar para hacer los llamados y para lidiar con los datos en formato json
import requests
from flask import  Flask, request, jsonify, render_template
import json



***

<a id="section_train"></a> 
<h3>Entrenamiento del modelo</h3>

[volver a TOC](#section_toc)

In [24]:
df = pd.read_csv('Data/co2_emissions_clean.csv')
df.dropna(inplace=True)

In [25]:
# Definimos los datos que vamos a pasar en el request
base=df.to_json() #dataframe en formato json
lista_predictores=df.drop(columns=['vehiculo_marca', 'vehiculo_modelo', 'vehiculo_tipo',
                                   'vehiculo_traccion','vehiculo_tipo_transmision','vehiculo_tipo_combustible', 
                                   'vehiculo_standard_emision', 'emision_CO2']).columns.values #features para entrenar el modelo
variable_target='emision_CO2' 
directory='/media/ignacio/DH/desafio-final' #dirección donde vamos a guardar el modelo  
name='predecir_consumo_Co2' #nombre modelo

url = 'http://localhost:5000/entrenar_modelo' #definimos la URL para hacer el llamado local con el endpoint correspondiente

In [26]:
# Hacemos el pedido y vemos los resultados.
modelo = requests.post(url,
                       json = {'base' : base,
                               'lista_predictores':list(lista_predictores),
                               'target':variable_target,
                               'directory':directory,
                               'name':name})
display(modelo.json())

{'r2_resultado_test': 0.8430373731777071,
 'r2_resultado_train': 0.892148002360325}

***

<a id="section_coef"></a> 
<h3>Visualización de Coeficientes</h3>

[volver a TOC](#section_toc)

In [27]:
#Definimos los datos que vamos a pasar en el pedido
dir_modelo='/media/ignacio/DH/desafio-final' #dirección donde se encuentra el modelo entrenado
nombre_modelo='predecir_consumo_Co2' #nombre del modelo entrenado
palette='hot' #paleta de colores para usar en el gráfico de barras
size=(25,8) #tamaño de la figura
predictores=list(lista_predictores) #lista con los features en el mismo orden en el que fueron pasados en el modelo
url = 'http://localhost:5000/plot_coeficientes' #definimos la URL para hacer el llamado local con el endpoint correspondiente

In [28]:
# Hacemos el pedido e imprimimos el output
coef = requests.post(url,
                     json = {'direccion' : dir_modelo,
                             'nombre_modelo':nombre_modelo,
                             'paleta_colores':palette,
                             'tamano':size,
                             'predictores':predictores})
print(coef.json())

{"coeficientes":{"consumo_extraurbano":14.598618527,"consumo_mixto":11.4344132914,"consumo_urbano":0.0380119041,"vehiculo_cilindrada":0.0005168963}}


***

<a id="section_predict"></a> 
<h3>Solicitud de predicción</h3>

[volver a TOC](#section_toc)

In [ ]:
#Definimos las variables que vamos a utilizar para el pedido
direccion_modelo='/media/ignacio/DH/desafio-final' #dirección donde se encuentra el modelo grabado
nombre_modelo='predecir_consumo_Co2' #nombre del modelo grabado

features=df.drop(columns=['vehiculo_marca', 'vehiculo_modelo', 'vehiculo_tipo',
                                   'vehiculo_traccion','vehiculo_potencia','lca_numero', 
                                   'vehiculo_id_motor','vehiculo_id_motor','fecha_firma',
                                   'ensayo_gei_numero','ensayo_gei_laboratorio','id_etiqueta',
                                  'vehiculo_tipo_transmision','vehiculo_tipo_combustible', 
                                   'vehiculo_standard_emision', 'emision_CO2']).iloc[3].to_json() #elegimos features de un jugador 
#cualquiera de la base para chequear que funcione bien el llamado
url = 'http://localhost:5000/prediccion' #definimos la URL para hacer el llamado local con el endpoint correspondiente


In [30]:
#Hacemos el llamado
pred = requests.get(url, 
                    params = {'direccion':direccion_modelo,
                              'nombre_modelo':nombre_modelo,
                              'features':[features]})
print(pred.json())

{'prediccion_co2': 193.9197967070415}
